In [144]:
from indeed import IndeedClient
import re
client = IndeedClient(publisher ="9124561496937265")
#Attribution: https://github.com/indeedlabs/indeed-python

In [23]:
from indeed import IndeedClient

client = IndeedClient("9124561496937265")

params = {
    'q' : "self-driving+car",
   # 'l' : "austin",
    'userip' : "1.2.3.4",
    'useragent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2)"
}

search_response = client.search(**params)

In [105]:
# Example of a single examples
this_job = search_response['results'][0]
#print(this_job)
#get jobkey to get more details
this_jobkey = this_job['jobkey']
#print(this_jobkey)

#Make a query on this jobkey
this_jobDetails = client.jobs(jobkeys = (this_jobkey, ))
#############################
######  JOB DETAILS  ########
#############################
job_title = this_jobDetails['results'][0]["jobtitle"]
job_postdate = this_jobDetails['results'][0]["date"]
job_employer = this_jobDetails['results'][0]["company"]
job_country = this_jobDetails['results'][0]["country"]
job_state = this_jobDetails['results'][0]["state"]
job_gps = {'long': this_jobDetails['results'][0]["longitude"], 'lat': this_jobDetails['results'][0]["latitude"] }
print(job_title+"\n")
print(job_postdate+"\n")
print(job_employer+"\n")
print(job_country+" ("+job_state+")\n")
print("long: {} | lat: {}".format(job_gps['long'], job_gps['lat']))

Deep Learning Performance Architect

Wed, 26 Apr 2017 00:07:31 GMT

NVIDIA

US (OR)

long: -122.8022 | lat: 45.486263


In [72]:
from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline

In [ ]:
#website parsing function

In [171]:
url = 'http://www.indeed.com/viewjob?jk='+str(this_jobkey)
#site = urllib.request.urlopen(url).read() # Connect to the job posting
#soup_obj = BeautifulSoup(site, "lxml")
try:
    site = urllib.request.urlopen(url).read() # Connect to the job posting
    soup_obj = BeautifulSoup(site, "lxml")
except:
    print('page not found')
    #return   # Need this in case the website isn't there anymore or some other weird connection problem 

spans = soup_obj.find_all('span', attrs={'id':"job_summary"})
for span in spans:
    print('found')

degrees = ['PhD', 'MS', 'Bachelor', 'PHD', 'BSc']
eduFields = ['CS', 'CE', 'EE', 'Physics']

domains = {'machine': ['learning'], 'Deep': ['Learning'], 'computer': ['architecture'], 'Algorithms':[], 
                'GPU': ['computing'], 'parallel': ['programming'], 'Computer': ['Vision']}

progSkills = ['C++', 'C', 'C#', 'Python', 'python', 'OpenCV', 'openCV']

#convert soup object to text
text = spans[0].get_text()


#Cleaning text
try:
    text = text.decode('unicode_escape').encode('ascii', 'ignore') 
except:                                 
    print('bog')

#Remove special characters
special_chars = "!@#$%^&*()[]{};:,./<>?\|`~=_"
for special_char in special_chars:
    text = text.replace(special_char, " ")
#remove newline
text = " ".join(text.split())

text = re.sub("[^a-zA-Z.+3]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                                # Also include + for C++
text_split = text.split()
print(text_split)

found
bog
['We', 'are', 'now', 'looking', 'for', 'a', 'Deep', 'Learning', 'Performance', 'Architect', 'NVIDIA', 'is', 'seeking', 'extraordinary', 'architects', 'to', 'develop', 'processor', 'and', 'system', 'architectures', 'that', 'accelerate', 'machine', 'learning', 'data', 'analytics', 'and', 'high', 'performance', 'computing', 'applications', 'This', 'position', 'offers', 'the', 'chance', 'to', 'create', 'a', 'meaningful', 'impact', 'in', 'a', 'dynamic', 'technology', 'focused', 'company', 'What', 'you', 'will', 'be', 'doing', 'As', 'a', 'member', 'of', 'our', 'deep', 'learning', 'architecture', 'team', 'you', 'will', 'craft', 'system', 'and', 'processor', 'architectures', 'to', 'extend', 'the', 'state', 'of', 'the', 'art', 'in', 'deep', 'learning', 'efficiency', 'and', 'performance', 'Prototype', 'key', 'deep', 'learning', 'and', 'data', 'analytics', 'algorithms', 'and', 'applications', 'Analyze', 'trade', 'offs', 'in', 'performance', 'cost', 'and', 'power', 'developing', 'analyti

In [175]:
job_fields = ['degrees', 'eduFields', 'domains', 'progSkills']

this_job = {}
#Initialize
for i in job_fields:
    this_job[i] = []

#better to scroll thru text once
for idx, word in enumerate(text_split):
    w_lower = word.lower()
    if word in degrees:
        if len(this_job['degrees'])==0:
            this_job['degrees'] = [word]
        else:
            this_job['degrees'].append(word)

    if word in eduFields:
        if len(this_job['eduFields'])==0:
            this_job['eduFields'] = [word]
        else:
            this_job['eduFields'].append(word)
    
    if w_lower in domains:
        next_word = text_split[idx+1]
        if next_word.lower() in domains[w_lower]:
            # word+next_word combination 
            if len(this_job['domains'])==0:
                this_job['domains'] = [w_lower+" "+next_word.lower()]
            else:
                domain = w_lower+" "+next_word.lower()
                if domain not in this_job['domains']:
                    this_job['domains'].append(domain)
    
    if word in progSkills:
        if len(this_job['progSkills'])==0:
            this_job['progSkills'] = [word]
        else:
            this_job['progSkills'].append(word) 
            

print(this_job)


#convert soup object to text
#text_split = text.lower().split()
#print(text_split)

{'progSkills': ['Python', 'C', 'C++'], 'degrees': ['MS', 'PhD'], 'eduFields': ['CS', 'EE', 'CE'], 'domains': ['machine learning', 'computer architecture', 'parallel programming']}
